In [ ]:
# ==========================================================
# ADAM-Next ULTRA-FAST DEMO (MRI) – ALL ALGOS, EPOCH = 1
# ==========================================================

!pip install -q opendatasets tensorflow catboost lightgbm pytorch-tabnet gradio matplotlib seaborn

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from tensorflow import keras
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
\# ---------------- DOWNLOAD DATASET ----------------
import opendatasets as od
od.download("https://www.kaggle.com/datasets/lukechugh/best-alzheimer-mri-dataset-99-accuracy", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: manginenijagadeesh
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lukechugh/best-alzheimer-mri-dataset-99-accuracy


100%|██████████| 71.5M/71.5M [00:00<00:00, 1.46GB/s]

In [ ]:
# ---------------- DATASET ----------------
DATASET_PATH = "/content/best-alzheimer-mri-dataset-99-accuracy"
# Correct the DATASET_PATH to point to the directory containing class folders
DATASET_PATH_CORRECTED = os.path.join(DATASET_PATH, "Combined Dataset") # Renamed to avoid confusion

IMG_SIZE = (160,160)   # smaller = faster

datagen = ImageDataGenerator(rescale=1./255)

data = datagen.flow_from_directory(
    DATASET_PATH_CORRECTED, # Use corrected path
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False # Keep false for consistent feature extraction order
)

class_names = list(data.class_indices.keys())
y_full = data.classes # Get all classes

# ---------------- FAST CNN FEATURE EXTRACTOR ----------------
base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(160,160,3))
x = GlobalAveragePooling2D()(base.output)
cnn = Model(base.input, x)

X_full = cnn.predict(data, verbose=0) # Extract features for all images

# 🔥 LIMIT DATA FOR FAST DEMO - Apply stratified sampling if MAX_SAMPLES is less than total
MAX_SAMPLES = 300 # Keep MAX_SAMPLES as defined by user

# Ensure there are multiple classes and enough samples to stratify
if len(np.unique(y_full)) > 1 and len(y_full) > MAX_SAMPLES:
    from sklearn.model_selection import StratifiedShuffleSplit
    # Use StratifiedShuffleSplit to get MAX_SAMPLES while maintaining class distribution
    # We want one split, and we want to select MAX_SAMPLES for our working dataset (X, y)
    sss = StratifiedShuffleSplit(n_splits=1, train_size=MAX_SAMPLES, random_state=42)
    for index_sample, _ in sss.split(X_full, y_full):
        X = X_full[index_sample]
        y = y_full[index_sample]
else:
    # If not enough samples for stratified split or only one class, use all available data
    # (after correcting DATASET_PATH, this branch should only hit if MAX_SAMPLES >= total samples)
    X = X_full
    y = y_full

# ---------------- SPLIT ----------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ---------------- AUTOENCODER (EPOCH = 1) ----------------
autoencoder = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(X_train.shape[1])
])
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=1, verbose=0)

X_train_enc = autoencoder.predict(X_train)
X_test_enc = autoencoder.predict(X_test)

# ---------------- MODELS ----------------
models = {
    "CatBoost": CatBoostClassifier(verbose=0),
    "LightGBM": LGBMClassifier(),
    "TabNet": TabNetClassifier(verbose=0),
    "Autoencoder+GB": GradientBoostingClassifier()
}

results = []

for name, model in models.items():
    if name == "Autoencoder+GB":
        model.fit(X_train_enc, y_train)
        preds = model.predict(X_test_enc)
    else:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds, average="weighted"),
        "Recall": recall_score(y_test, preds, average="weighted"),
        "F1": f1_score(y_test, preds, average="weighted")
    })

results_df = pd.DataFrame(results)
best_model_name = results_df.sort_values("F1", ascending=False).iloc[0]["Model"]
best_model = models[best_model_name]

# ---------------- BAR GRAPH ----------------
def bar_plot():
    fig, ax = plt.subplots()
    sns.barplot(x="Model", y="F1", data=results_df, ax=ax)
    plt.xticks(rotation=30)
    return fig

# ---------------- GRADIO PREDICTION ----------------
def predict(img):
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    feat = cnn.predict(img)
    feat = scaler.transform(feat)

    if best_model_name == "Autoencoder+GB":
        feat = autoencoder.predict(feat)

    pred = best_model.predict(feat)[0]
    label = "Alzheimer Detected" if pred != 0 else "No Alzheimer Detected"

    pie_fig, ax = plt.subplots()
    ax.pie([1,0], labels=["Prediction",""], autopct='%1.1f%%')
    plt.title(label)

    return label, bar_plot(), pie_fig, results_df.to_string(index=False)

# ---------------- GRADIO APP ----------------
app = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Textbox(label="Diagnosis"),
        gr.Plot(label="Model Comparison (F1 Score)"),
        gr.Plot(label="Prediction Confidence"),
        gr.Textbox(label="Accuracy / Precision / Recall / F1")
    ],
    title="ADAM-Next Fast Alzheimer MRI Detection (Demo)",
    description="CNN + CatBoost + LightGBM + TabNet + Autoencoder+GB | Epoch = 1"
)

app.launch(debug=True)


Found 11519 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[LightGBM] [Info] Number of positive: 200, number of negative: 25
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 52997
[LightGBM] [Info] Number of data points in the train set: 225, number of used features: 1018
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.888889 -> initscore=2.079442
[LightGBM] [Info] Start training from score 2.079442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://08859f5600e43a114e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
